# Stock market Indicator Using Klangoo

### The stock prices are directly related to people opinion and trust. Sentiment Analysis can be used to determine the overall opinion for a certain stock. 

### A fast answer can be to apply sentiment analysis on every text contaning the word(s) we are interrested in.

### But, It is not accurate to study every text containing the keyword we are looking for, si how de we know if the domain of the text is  the same domain we we are interrested in (Buisness, Technology, ...) or the text is about a completely different domain and citing this word as an example while speaking about another thing??  

### How do we know if a certain text is directly related to the stock we are interrested in (i.e.: the stock is the main subject or one of the main subjects of the text) and not cited as a secondary subject in the text?

## KLANGOO can be the answer

### Klangoo can be used to determine, for each text (twitter tweet or a newspaper article) if the domain and the main subject of the text (entities) are what we are interrested in. 


### I used a dataset already containng the sentiment analysis results in the first part. we used a stock market tweets from kaggle, (https://www.kaggle.com/datasets/sohelranaccselab/stock-market-tweets-data-sentiment-analysis ).
### Each row of the csv file contains a tweet details, the tweet initial text and a sentiment analysis result (positive, negative, neutral) .
### After sending the tweet as a requeest to Klangoo, we process the results: if the main category and the main entities of the tweet are appropriate: the tweet will be processed. Otherwise it will be neglected.
### A final sentiment analysis will be given to the wanted entity. If it is negative, this could be an indicator that the  stock is going down (maybe, nothing is 100% sure in stock market), and vice versa.

### The result will be a simple advice of selling, or keeping / buying a certain stock according to the general public opinion. Also a final grade over 5 will be given with the number of tweets analysed to give a certain inficator to the user.

### In the secaond part (real use case) an API was used to collect articles from different source, in a certain history range, to make the sentiment analysis calcualtion on.


### Define a function that will be used in the code

In [150]:
def have_common(x,y):
   common = False
   for value in x:
      if value in y:
         common= True
   return common

### Importing Libraries and reading the csv file a Pandas dataFrame

In [151]:
from klangooclient.MagnetAPIClient import MagnetAPIClient
import pandas as pd
import json
import string

df = pd.read_csv("tweets_labelled.csv") #Read  csv file containig 5000 tweets and the sentiment of each tweet
                                                       # The csv file from kaggle contains alreade the sentiment analysis 
df = df[:5] #only 30 rows were processed 
print()

### Prepare connection strings and secret key

In [152]:
ENDPOINT ='https://nlp.klangoo.com/Service.svc'
CALK = '155b99fc-758c-40c8-a8ef-5d4eb9fba616'
SECRET_KEY = 'wTMeUVHL38vGrdDBbSdSFBQOWFchPTaUnIFZBJS4'

client = MagnetAPIClient(ENDPOINT, CALK, SECRET_KEY)

### Specify wanted entities and categories to search for

In [159]:
wanted_stock = "Amazon stock AMZN"
wanted_entities = ['amazon', 'amzn'] #Stock(s) interrested in, This option is subjective to the user.
wanted_categories = [ 'science','general','business', 'politics','technology'] #Only these categoreis will be taken into considertion

### Code

In [160]:
n = len(df)
ctr = 0
grade_sum = 0
final_grade = 0

for i in range(n): #read all the tweets from  the dataFrame
    text = df.iloc[i,2] #get text from dataFrame
    text = text
    sentiment = df.iloc[i,3]   #get sentiment analysis result from dataFrame
        
    request =  { 'text' : text, 'lang' : 'en', 'format' : 'json' }
    
    rsp = client.callwebmethod('ProcessDocument', request, 'POST') #Use Klangoo to get all entities and categories of tweet
    info = json.loads(rsp) #transform to Dictionary
    info.fromkeys
    print("request nb:", i , "done")
    print("Text for request nb:", i, "is: ", text)
    print("Sentiment for request nb:", i, "is: ", sentiment)

    entities_nb = len(info['document']['entities'])
    categories_nb = len(info['document']['categories']) 
    
    text_entities = []   # get the key topics from text and append to this list
    text_categories = []   # get the categories from text and append to this list 
    
    for j in range(entities_nb):
        text_entities.append(info['document']['entities'][j]['token'].lower())#adding categoreis from klangoo response to  text_categories
    
    for j in range(categories_nb):
        text_categories.append(info['document']['categories'][j]['name'].lower())#adding categoreis from klangoo response to  text_categories
    
    print("Entities for request nb:", i, "are: ", text_entities)
    print("Categories for request nb:", i, "are: ", text_categories, "\n")

    check_category = have_common(wanted_categories, text_categories ) #check existecne of an item of text_categories in wanted_category 
    check_entities = have_common(wanted_entities, text_entities )
    
    if check_category  and check_entities: # if the category and topics of the text are in what we are interrested in 
        if sentiment == 'positive':
            grade_sum += 1   
        elif sentiment == 'negative':
            grade_sum -= 1
        #else is: stays zero 
        ctr += 1
           
  

if ctr != 0:
    final_grade = grade_sum / ctr

    if final_grade <0:
        print("The " , wanted_stock  ,"have negative Semantic Analysis in the given tweets, it could be an indicator \
        that this stock does not have community trust in the dtermined time.\n  After \
        processing ", ctr , " tweets, the term Netfilx or NFLX got a scrore of : " , final_grade, " on 1 .")
    elif final_grade >0:
        print("The ",wanted_stock, "  have a positive Semantic Analysis in the given tweets, it could be an indicator \
        that this stock have community trust in the dtermined time.\n  After \
        processing ", ctr , " tweets, the term Netfilx or NFLX got a scrore of : " , final_grade, " on 1 .")
    else:
        print("After analysing the specifeid texts (tweets), The ", wanted_stock, " have a neutral Semantic Analysis. ")  

request nb: 0 done
Text for request nb: 0 is:  RT @vincent13031925: Biggest mistakes of the century: From before  Apple AAPL is  only a PC maker  Amazon AMZN is only a bookstore Goo…
Sentiment for request nb: 0 is:  negative
Entities for request nb: 0 are:  ['amazon', 'apple', 'rt']
Categories for request nb: 0 are:  ['technology', 'business'] 

request nb: 1 done
Text for request nb: 1 is:  RT @BarrySchwartzBW: Carl Icahn once owned 10% of Netflix. That stake today would have been worth about 24B. He also had 52m shares of App…
Sentiment for request nb: 1 is:  neutral
Entities for request nb: 1 are:  ['carl icahn', 'netflix']
Categories for request nb: 1 are:  ['business', 'technology'] 

request nb: 2 done
Text for request nb: 2 is:  RT @gzbenso: Netflix leads its rivals in original TV shows by a wide margin in both quantity and quality  according to new data analysis ht…
Sentiment for request nb: 2 is:  positive
Entities for request nb: 2 are:  ['netflix']
Categories for request nb:

## Second Part: Real Use case (Bitcoin)

### Now, after we tried our idea on a dataset, let's try our idea on a real use case. We will get news articles from different newspapers and apply the sentiment analysis and use klangoo to make a final indicator only for the stocks we are interrested in.

### The newsapi is used here to provide articles from different sources (Better than searching in hundreds of websites) containig a specific word and in a certain date range (from -to), in ceratin websites we specify or over 80,000 websites.

### But it is not logical to take all articles containng this word into consideration. So we will use Klangoo to know the main entities and the domain of the article, then we can apply sentiment analysis.


###

## Bitcoin stock Indicator between 01/07/2022 and 13/07/2022

In [227]:
import requests

#get all the articles from .. to .. containin the words realted to bicoin
url = ('https://newsapi.org/v2/everything?' 
       'q=Bitcoin USD& Or q=BTC-USD& or q=bitcoin& or q=Bitcoin& or q=BTC& or q=btc&'  #serach for articles containng ... word
       'from=2022-07-01&'     #search article in the last week 
       'to=2022-07-13&'
       'language=en&'
       'sortBy=popularity&'
       'apiKey=42f6576050c04962b5556e40825857df')

response = requests.get(url)
print("Request successful")

Request successful


In [245]:
pip install newsdataapi

Note: you may need to restart the kernel to use updated packages.


In [265]:
from newsdataapi import NewsDataApiClient

api = NewsDataApiClient(apikey='pub_9173b9c5020815c947c4ebd4e19094a1b11d')

# You can pass empty or with request parameters {ex. (country = "us")}

response = api.news_api( q= "bitcoin OR Bitcion OR BTC-USD OR Bitcoin USD OR BTC Or btc" , language='en')
#Bitcoin USD& Or q=BTC-USD& or q=bitcoin& or q=Bitcoin& or q=BTC& or q=btc&

#https://newsdata.io/api/1/news?apikey=YOUR_API_KEY&q=social%20OR%20pizza

In [266]:
print(response)

{'status': 'success', 'totalResults': 5279, 'results': [{'title': '3 Reasons Why ‘Seeking Sister Wife’ Fans Suspect Bert Dumped Garrick Merrifield', 'link': 'https://www.cheatsheet.com/entertainment/3-reasons-why-seeking-sister-wife-fans-suspect-bert-dumped-garrick-merrifield.html/', 'keywords': ['Entertainment', 'TV', 'seeking sister wife'], 'creator': ['Julia Odom'], 'video_url': None, 'description': "Here are the 3 main reasons why 'Seeking Sister Wife' fans suspect that Bert and Garrick are done. Between the lack of contact and the visa, is Bert done with him?", 'content': 'It’s been three years since Seeking Sister Wife star, Garrick Merrifield began courting Roberta ‘Bert’ Pache as a second wife and she’s still not in the United States. Many Seeking Sister Wife fans think Bert has already given up on the Merrifields. Here are the 3 reasons why Seeking Sister Wife fans think Bert already dumped Garrick and his wife, Dannielle Merrified. Dannielle Merrifield, Garrick Merrifield, an

In [221]:
#determine the wanted entity and categories

wanted_stock_2 = "Amazon sotck AMZN"
wanted_entities_2 = ['amazon', 'amzn'] #Stock(s) interrested in, This option is subjective to the user.
wanted_categories_2 = ['general','business', 'politics','technology']#Only these categoreis will be taken into considertion


In [182]:
f=open("C:\\Users\\Lenovo\\Desktop\\Project\\results.txt", "a")
f.write("Now the file has more content!")
f.close()


In [222]:
l = 0
title = response['articles'][l]['title']
desc = response['articles'][l]['description']
text = title +" "+ desc
text.strip()

'‘It’s Ruined Me’: Voyager Customers Fear Life Savings Gone After Crypto Firm’s Bankruptcy Voyager customers are struggling with depression and fearing the worst after the crypto brokerage suspended withdrawals and declared bankruptcy. “I was dumb enough to trust them,” one said'

In [229]:
print(len(response['articles']))

100


In [244]:
for i in range(10,30,1):
    print("title: " , response['articles'][i]['title'])
    print("desc: " , response['articles'][i]['description'])
    print("content: " , response['articles'][i]['content'])
    print("====================")

title:  Stablecoins, Riskycoins, and Dodgycoins
desc:  Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the economic functions that various digital tokens perform, rathe…
content:  Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the e… [+1903 chars]
title:  Bitfarms In Distress But Still Functional
desc:  Bitfarms' liquidation of its Bitcoin reserves wasn't a surprise. Read why we believe BITF stock is fairly priced but IREN offers a better margin of safety.
content:  Jay Yuno/E+ via Getty Images
Introduction
 A lot has happened since our last Bitfarms (NASDAQ:BITF) coverage 2 quarters ago. The recent headline news was BITF forgoing its HODL strategy to accumula… [+12765 chars]
title:  Soluna: The Bu

In [184]:
text_2 = response['articles'][2]['content']
f=open("C:\\Users\\Lenovo\\Desktop\\Project\\results.txt", "a")

for i in range(len(response['articles'])):
    f.write(response['articles'][i]['content'])
    f.write(" +++++++++++++++++ \n")

    
f.close()

In [233]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

nb = len(response['articles'])
temp_nb = 7 #For testing only
ctr_2 = 0
grade_sum_2 = 0
final_grade_2 = 0

for i in range(10,13,1):
    #Since the API give only 200 chars in contnet, this is not acceptable
    # a good idea would be to use the title and the description as a subsitute since they contion most of the meaning.
    title =""# response['articles'][i]['title']
    description = response['articles'][i]['description']
    text_2 = title +". "+ description
    text_2 = text_2.strip()
    text_2.replace('\n', ' ').replace('\r', '')

    request_2 =  { 'text' : text_2, 'lang' : 'en', 'format' : 'json' }
    rsp_2 = client.callwebmethod('ProcessDocument', request_2, 'POST') #Use Klangoo to get all entities and categories of tweet
    info_2 = json.loads(rsp_2) #transform from Bytes to Json format
    info_2.fromkeys
    
    print("Request nb:",i,"is successful.")
    print("Text nb:", i, "is: ", text_2)
    entities_nb_2 = len(info_2['document']['entities'])
    categories_nb_2 = len(info_2['document']['categories']) 
    
    text_entities_2 = []   # get the key topics from text and append to this list
    text_categories_2 = []   # get the categories from text and append to this list 
    
    for j in range(entities_nb_2):#adding entites from klangoo response to  text_entities_2 list
        text_entities_2.append(info_2['document']['entities'][j]['token'].lower() ) 
    
    for j in range(categories_nb_2):#adding categoreis from klangoo response to  text_categories_2
        text_categories_2.append(info_2['document']['categories'][j]['name'].lower() ) # append the key categories to the list
    
    print("Entities for Text nb:", i, "are: ", text_entities_2)
    print("Categories for Text nb:", i, "are: ", text_categorie_2, "\n")
    print("-----------------")
    print()
    check_category_2 = have_common(wanted_categories_2, text_categories_2 ) #check existecne of an item of text_categories in wanted_category 
    check_entities_2 = have_common(wanted_entities_2, text_entities_2 )
    
    if check_category_2  and check_entities_2: #If the article category and entity are the wanted ones, we apply S.A.
        ctr_2 += 1
        score_2 = analyser.polarity_scores(text2)
        grade_sum_2 += score_2['compound'] #compound element of the polarity score determine if positive >0.5 or negative <-0.5


        
if ctr_2 > 0:
    final_grade_2 = grade_sum_2 / ctr_2

    if final_grade_2 < -0.5: #check the condition neg or pos and total score in NLTK
        print("The " , wanted_stock_2  ,"have negative Semantic Analysis in the given tweets,it could be an indicator \
        that this stock does not have community trust in the dtermined time.\
        After processing ", ctr , " tweets, the term Netfilx or NFLX got a polarity scrore of : " , final_grade_2, " on  .")
    elif final_grade > 0.5:
        print("The ",wanted_stock_2, "  have a positive Semantic Analysis in the given tweets,it could be an indicator \
        that this stock have community trust in the dtermined time.\
        After processing ", ctr , " tweets, the term Netfilx or NFLX got a scrore of : " , final_grade_2, " on  .")
    else:
        print("After analysing the relative articles, the ", wanted_stock_2, " have a neutral Semantic Analysis.")          
    

    


Request nb: 10 is successful.
Text nb: 10 is:  . Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the economic functions that various digital tokens perform, rathe…
Entities for Text nb: 10 are:  []
Categories for Text nb: 10 are:  [] 

-----------------

Request nb: 11 is successful.
Text nb: 11 is:  . Bitfarms' liquidation of its Bitcoin reserves wasn't a surprise. Read why we believe BITF stock is fairly priced but IREN offers a better margin of safety.
Entities for Text nb: 11 are:  ['bitcoin']
Categories for Text nb: 11 are:  [] 

-----------------

Request nb: 12 is successful.
Text nb: 12 is:  . Soluna is priced fairly but it's Bitcoin-based business model might backfire. Why do I not recommend SLNH stock? Click here to find out.
Entities for Text nb: 12 are:  ['bitcoin']
Categories for Text nb: 12 are:  [] 

-----------------



#### Analyse sentiment analysis with NLTK library

In [209]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
sentiment_result = analyser.polarity_scores("Its time to start stacking bitcoin again.\
Thats the message from indicators tracking tokens sold by miners and comparing the \
cryptocurrencys market value to its fair value.uell Multiple ") 
     
print("Overall sentiment dictionary is : ", sentiment_result)
print("sentence was rated as ", sentiment_result['neg']*100, "% Negative")
print("sentence was rated as ", sentiment_result['neu']*100, "% Neutral")
print("sentence was rated as ", sentiment_result['pos']*100, "% Positive")
print("sentence compund is  ", sentiment_result['compound'], "")

print("Sentence Overall Rated As", end = " ")
 
# decide sentiment as positive, negative and neutral
if sentiment_result['compound'] >= 0.05 :
    print("Positive")
 
elif sentiment_result['compound'] <= - 0.05 :
        print("Negative")
 
else :
        print("Neutral")

    

Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.842, 'pos': 0.158, 'compound': 0.5719}
sentence was rated as  0.0 % Negative
sentence was rated as  84.2 % Neutral
sentence was rated as  15.8 % Positive
sentence compund is   0.5719 
Sentence Overall Rated As Positive


In [234]:
response['articles'][10]['description']

'Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the economic functions that various digital tokens perform, rathe…'

In [236]:
s = " Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the economic functions that various digital tokens perform, rathe…\
asdasdasd asd asd asdf asdf asdf\
"

In [237]:
s


' Recent price implosions have shown that not all so-called stablecoins are so stable after all. As policymakers draft regulations to bring some order to the crypto industry, they should focus on the economic functions that various digital tokens perform, rathe…asdasdasd asd asd asdf asdf asdf'